In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/shl-intern-hiring-assessment-2025'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_49.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_67_1.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_90.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_77.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_20_1.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_66.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_54.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_106_1.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_42.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_81.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_72.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_107.wav
/kaggle/input/shl-in

In [2]:
!pip install -q openai-whisper
!pip install -q nltk scikit-learn pandas tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 17.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.6 MB/

In [3]:
import os
import whisper
import pandas as pd
import nltk
from nltk import word_tokenize, pos_tag
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from tqdm import tqdm

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

model = whisper.load_model("base")

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
100%|███████████████████████████████████████| 139M/139M [00:02<00:00, 62.1MiB/s]


In [4]:
train_df = pd.read_csv("/kaggle/input/shl-intern-hiring-assessment-2025/dataset/csvs/train.csv")
test_df = pd.read_csv("/kaggle/input/shl-intern-hiring-assessment-2025/dataset/csvs/test.csv")
# sample_submission = pd.read_csv("/kaggle/input/shl-dataset/dataset/sample_submission.csv")
train_df.head()

,filename,label
0,audio_173,3.0
1,audio_138,3.0
2,audio_127,2.0
3,audio_95,2.0
4,audio_73,3.5


In [5]:
def extract_features(audio_path):
    result = model.transcribe(audio_path, fp16=False)
    text = result['text'].strip()
    
    tokens = word_tokenize(text)
    pos_tags = pos_tag(tokens)

    total_words = len(tokens)
    num_nouns = sum(1 for word, tag in pos_tags if tag.startswith('NN'))
    num_verbs = sum(1 for word, tag in pos_tags if tag.startswith('VB'))
    num_adjs = sum(1 for word, tag in pos_tags if tag.startswith('JJ'))
    avg_word_len = sum(len(word) for word in tokens) / total_words if total_words else 0

    return {
        'text': text,
        'total_words': total_words,
        'num_nouns': num_nouns,
        'num_verbs': num_verbs,
        'num_adjs': num_adjs,
        'avg_word_len': avg_word_len
    }

In [6]:
def get_all_audio_paths(base_filename, base_dir):
    """Generates all potential audio file paths for a base filename."""
    paths = []
    # Check for the base filename (e.g., audio_173.wav)
    paths.append(os.path.join(base_dir, f"{base_filename}.wav"))
    
    # Check for variations (e.g., audio_77_2.wav). Max variation seems to be _2.
    for suffix in ['_1', '_2']:
        paths.append(os.path.join(base_dir, f"{base_filename}{suffix}.wav"))
        
    # Return only the paths that actually exist in the file system
    return [path for path in paths if os.path.exists(path)]

# ... (Previous imports and definition of extract_features)

# --- Updated Execution Block ---

train_features = []
base_dir = "/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/train/"

# Before the main loop, fix the NLTK issue:
nltk.download('averaged_perceptron_tagger_eng')

for i, row in tqdm(train_df.iterrows(), total=len(train_df)):
    base_filename = row['filename']
    label = row['label']
    
    # 1. Find all relevant audio files for this entry
    audio_paths = get_all_audio_paths(base_filename, base_dir)
    
    if not audio_paths:
        # If no audio files match the naming conventions, skip this row
        print(f"⚠️ Could not find any audio files for base ID: {base_filename}. Skipping.")
        continue

    # 2. Process each found audio file
    for audio_path in audio_paths:
        try:
            features = extract_features(audio_path) 
            features['label'] = label
            # Store the unique filename (including suffix) for tracking
            features['full_filename'] = os.path.basename(audio_path)
            train_features.append(features)
        except Exception as e:
            print(f"❌ Failed to process {os.path.basename(audio_path)}: {e}")

# Convert to DataFrame
train_features_df = pd.DataFrame(train_features)
print(train_features_df.head())

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
100%|██████████| 409/409 [10:30<00:00,  1.54s/it]

                                                text  total_words  num_nouns  \
0  My favorite place to visit will be Japan, beca...          110         17   
1  I love to reading on my hobby such reading. Em...          159         41   
2  I love to read him and my hobbies as duty. Emm...          162         41   
3  My favorite place to visit is Mullah Itis near...           68         16   
4  Diisia asrali tied the temple, dhocit puriadan...           16         12   

   num_verbs  num_adjs  avg_word_len  label    full_filename  
0         21         7      3.781818    3.0    audio_173.wav  
1         29        12      4.459119    3.0    audio_138.wav  
2         25        12      4.395062    3.0  audio_138_2.wav  
3         10         6      3.838235    2.0    audio_127.wav  
4          1         1      5.062500    2.0  audio_127_2.wav  


In [7]:
X = train_features_df[['total_words', 'num_nouns', 'num_verbs', 'num_adjs', 'avg_word_len']]
y = train_features_df['label']

from sklearn.ensemble import RandomForestRegressor

model_rf = RandomForestRegressor(random_state=42)
model_rf.fit(X, y)

RandomForestRegressor(random_state=42)

In [8]:
y_pred = model_rf.predict(X)
mse = mean_squared_error(y, y_pred)
print(f"Train MSE: {mse:.4f}")

Train MSE: 0.0704


In [9]:
test_features = []
# 1. Correct the base path to match the directory structure from your first cell's output
base_dir_test = "/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/"

# Also need to define 'extract_features' or ensure it's available from a previous cell.
# Assuming 'extract_features' is the function that handles transcription and feature calculation.

for i, row in tqdm(test_df.iterrows(), total=len(test_df)):
    audio_filename_base = row['filename']
    
    # 2. Append the missing .wav extension
    audio_filename = f"{audio_filename_base}.wav" 
    audio_path = os.path.join(base_dir_test, audio_filename) 
    
    try:
        if not os.path.exists(audio_path):
            # Check for alternative files like 'audio_XX_1.wav' if the base file is missing
            alternative_found = False
            for suffix in ['_1', '_2']:
                alt_path = os.path.join(base_dir_test, f"{audio_filename_base}{suffix}.wav")
                if os.path.exists(alt_path):
                    audio_path = alt_path
                    alternative_found = True
                    break
            
            if not alternative_found and not os.path.exists(audio_path):
                print(f"⚠️ Missing required audio file: {audio_filename_base} (and variants). Skipping.")
                continue

        features = extract_features(audio_path)
        features['filename'] = audio_filename_base  # Store the base filename for merging/submission
        test_features.append(features)
        
    except Exception as e:
        print(f"❌ Failed to process {audio_filename}: {e}")

# If the NLTK error still occurs, run the NLTK download line again:
# nltk.download('averaged_perceptron_tagger_eng')

100%|██████████| 197/197 [04:03<00:00,  1.24s/it]


In [10]:
test_feat_df = pd.DataFrame(test_features)
X_test = test_feat_df[['total_words', 'num_nouns', 'num_verbs', 'num_adjs', 'avg_word_len']]

In [11]:
preds = model_rf.predict(X_test)
submission = pd.DataFrame({'filename': test_df['filename'], 'label': preds})
submission.to_csv("submission.csv", index=False)
submission.head(100)

,filename,label
0,audio_141,2.540
1,audio_114,2.505
2,audio_17,2.655
3,audio_76,3.020
4,audio_156,2.940
...,...,...
95,audio_11,2.810
96,audio_63,2.440
97,audio_101,3.405
98,audio_144,2.610


In [12]:
submission

,filename,label
0,audio_141,2.540
1,audio_114,2.505
2,audio_17,2.655
3,audio_76,3.020
4,audio_156,2.940
...,...,...
192,audio_107,2.860
193,audio_15,2.805
194,audio_93,2.450
195,audio_31_1,2.745


10. Conclusion
We developed a simple, fast, and efficient Grammar Scoring Engine using Whisper transcription and basic rule-based features from the transcript. The model performs reasonably well on train data and can be improved further by:

Using more linguistic features like grammatical errors, parse trees, etc.
Incorporating BERT embeddings or transformer-based text models
Fine-tuning Whisper to your data
Using end-to-end audio-to-score models
This serves as a great baseline to build upon for the final Kaggle submission.